In [5]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Let's load our dataset

The `read_csv` method from pandas is our friend here, it keeps things nice and simple, it can be configurable for lot's of options, but the default settings work for us here. The one argument we specify is the URL where the file is hosted. You could visit this URL, download the CSV, and explore it in Excel if you want.

[pandas read_csv documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [4]:
df = pd.read_csv('https://storage.googleapis.com/pick-n-mix-1/convertcsv.csv')

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32844 entries, 0 to 32843
Data columns (total 6 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   LSOA code (2011)                            32844 non-null  object
 1   LSOA name (2011)                            32844 non-null  object
 2   Local Authority District code (2019)        32844 non-null  object
 3   Local Authority District name (2019)        32844 non-null  object
 4   Index of Multiple Deprivation (IMD) Rank    32844 non-null  int64 
 5   Index of Multiple Deprivation (IMD) Decile  32844 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.5+ MB
None


,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Rank,Index of Multiple Deprivation (IMD) Decile
0,E01000001,City of London 001A,E09000001,City of London,29199,9
1,E01000002,City of London 001B,E09000001,City of London,30379,10
2,E01000003,City of London 001C,E09000001,City of London,14915,5
3,E01000005,City of London 001E,E09000001,City of London,8678,3
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,14486,5


## Validation

Let's make sure that the data meets some of our expectations. The `Index of Multiple Deprivation (IMD) Rank` column is ranking, therefore no two LSOA locations should have the same ranking. Let's check if that assumption holds true?

In [ ]:
s

## Filtering

## Simple Transformation